In [132]:
# Defining a function which will take a word and will correct its unicode representation
def unicode_correction(word):
    unicode_word = []
    length = len(word)
    for i in range(length):
        if word[i] in punctuation_characters:
            continue
        if word[i]=='्':
            continue
        if word[i] in List_swaras:
            unicode_word.append(word[i])
        elif word[i] in List_matras:
            unicode_word.append(mapping_dict[word[i]])
        elif i!=length-1 and word[i+1] in List_matras:
            unicode_word.append(word[i]+'्')
        elif i!=length-1 and word[i+1] in List_swaras:
            unicode_word.append(word[i]+'्')
        else:
            if i != length-1 and word[i+1] == '्':
                unicode_word.append(word[i]+'्')
                continue;
            unicode_word.append(word[i]+'्')
            unicode_word.append('अ')
    return unicode_word

# Defining a function which will remove unnecessory punctuation signs
def clean(token_list):
  newlist=[]
  for tokens in token_list:
    str=""
    for ch in tokens:
      if ch not in punctuation_characters:
        str+=ch
    newlist.append(str)
  return newlist
# defining a function to break text into list of words using regular expression
def extract_words(text):
    pattern = r'[\u0900-\u097F]+'
    words = re.findall(pattern, text)
    return words

# defining function to break text into list of characters
def break_text(text):
    words = extract_words(text)
    broken_text = []
    for word in words:
        broken_word = unicode_correction(word)
        broken_text.append((word, broken_word))
    return broken_text

# defining function to load corpus from file
def load_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Reading only the first 1 lines which we can change anytimme
        text = file.read()
    return text

# Creating bigrams from a given Unicode word and returning list of bigram extracted from it
def create_bigrams(unicode_word):

    bigram_word = [unicode_word[i] + unicode_word[i+1] for i in range(len(unicode_word)-1)]
    return bigram_word

def create_syllable_bigrams(unicode_word):
    # Generate syllable bigrams
    syllable_bigram_word = [unicode_word[i] + unicode_word[i+1] for syllable in unicode_word for i in range(len(syllable)-1)]

    return syllable_bigram_word

# defining function to calculate bigram frequencies
def calculate_bigram_frequencies(text):
    bigram_freq = {}
    for _, unicode_word in text:
        for i in range(len(unicode_word) - 1):
            bigram = unicode_word[i] + unicode_word[i+1]
            if bigram in bigram_freq:
                bigram_freq[bigram] += 1
            else:
                bigram_freq[bigram] = 1
    sorted_bigram_freq = sorted(bigram_freq.items(), key=lambda x: x[1], reverse=True)
    return sorted_bigram_freq

def form_syllable(unicode_word_list):
    syllable_list = []
    for word in unicode_word_list:
        syllable = ""
        temp = []
        i = 0
        while i < len(word):
            char = word[i]
            if char in List_vyanjan_halant:
                if char == '्' and syllable.endswith('|'):  # for specific case of '|'
                    syllable += char
                elif i+1 < len(word) and word[i+1] in List_matras:
                    syllable += char + word[i+1]
                    i += 1  # Skip the next character (matra) as it's already included
                else:
                    if syllable:
                        temp.append(syllable)
                    syllable = char
            else:
                syllable += char
            i += 1

        if syllable:  # Append the remaining syllable
            temp.append(syllable)
        if temp:
            syllable_list.append(temp)
    return syllable_list

# defining function to calculate characters frequencies
def calculate_character_frequencies(broken_text):
    character_frequency = {}
    for _, unicode_word in broken_text:
        for char in unicode_word:
            if char in character_frequency:
                character_frequency[char] += 1
            else:
                character_frequency[char] = 1
    sorted_characters = sorted(character_frequency.items(), key=lambda x: x[1], reverse=True)
    return sorted_characters

# defining function to calculate syllable bigram frequencies
def calculate_syllable_frequencies(syllable_list):
    syllable_freq = {}
    for syllable in syllable_list:
        for sub_syllable in syllable:
            if sub_syllable in syllable_freq:
                syllable_freq[sub_syllable] += 1
            else:
                syllable_freq[sub_syllable] = 1
    sorted_syllables = sorted(syllable_freq.items(), key=lambda x: x[1], reverse=True)
    return sorted_syllables

In [ ]:
import re

# Defining the Unicode mapping dictionary
# List of hindi matras and swaras and vyanjan
List_matras = ['ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ै', 'ो', 'ौ', 'ं', 'ः', 'ँ']
List_swaras = ['अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ','अँ']
#string of punctuation_characters we need to avoid
punctuation_characters = "##▁#।▁,?!.:;‘’“”-…()▁"
Alphabet_eng =['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
List_vyanjan = ['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'क्ष', 'त्र', 'ज्ञ']
List_vyanjan_halant = [char + '्' for char in List_vyanjan]
# dictionary of mapping of matras to swaras
mapping_dict = {"ा": "आ", "ि": "इ", "ी": "ई", "ु": "उ", "ू": "ऊ", "े": "ए", "ै": "ऐ", "ो": "ओ", "ौ": "औ", "ं": "अं", "ृ": "ऋ", "ॉ": "ऑ", "्": "", "ः": "ः", 'ँ': "अँ"}
# dictionary of swaras to matras aka reverse of mapping_dict
reverse_mapping_dict = {'आ': 'ा', 'इ': 'ि', 'ई': 'ी', 'उ': 'ु', 'ऊ': 'ू', 'ए': 'े', 'ऐ': 'ै', 'ओ': 'ो', 'औ': 'ौ', 'अं': 'ं', 'ऋ': 'ृ', 'ऑ': 'ॉ','ॅ्':'', 'ः': 'ः','\n्':'', 'अँ': 'ँ','ळ्':'','अ': '', '़्': '','ृ्':'','ॉ्':''}
Alphabet_eng =['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

# Loading text from corpus file
corpus_file_path = "sample.txt"
text = load_corpus(corpus_file_path)

# Breaking the text into list of characters
broken_text = break_text(text)

# Printing the broken text along with the original words
for word, unicode_word in broken_text:
    print(f"Extracted Word :- {word} :-\n{unicode_word}\n")



In [131]:
import re

# Defining the Unicode mapping dictionary
# List of hindi matras and swaras and vyanjan

# Loading text from corpus file
corpus_file_path = "sample.txt"
text = load_corpus(corpus_file_path)

# Breaking the text into list of characters
broken_text = break_text(text)
print("bt",broken_text)
# Initialize lists to store Unicode-corrected words
unicode_word_list = []

# Printing the broken text along with the original words, syllables, and bigrams
for word, unicode_word in broken_text:
    unicode_word_list.append(unicode_word)
    bigram_word = create_bigrams(unicode_word)
    syllable_list = form_syllable([unicode_word])  # Call form_syllable for the current word
    syllable_list_bigrams = create_bigrams(syllable_list)

    # print(f"Extracted Word: {word}:")
    # print(f"Unicode-corrected Characters: {unicode_word}")
    # print(f"Unicode Bigram: {bigram_word}")
    # print(f"Syllables: {syllable_list}")
    # print(f"Syllables Bigram: {syllable_list_bigrams}\n")

print("uwl",unicode_word_list)
# Calling the form_syllable function
syllable_list = form_syllable(unicode_word_list)

# Printing the syllables in a similar format as characters and bigrams
syllable_bigram_list = [syllable_list[i]+syllable_list[i+1] for i in range(len(syllable_list)-1)]
# Calculating the frequency of each character

# Sorting the characters by frequency in descending order as we need the most freq char on top
sorted_characters = calculate_character_frequencies(broken_text)
# Calculating bigram frequencies
bigram_freq = calculate_bigram_frequencies(broken_text)

# Printing the top 20 unicode corrected characters and their frequencies
print("Top 20 characters:")
for char, frequency in sorted_characters[:20]:
    print(f"Character: {char}, Frequency: {frequency}")

# Print the top 20 bigram and their frequencies
print("\nTop 20 bigram frequencies:")
for bigram, frequency in bigram_freq[:20]:
    print(f"Bigram: {''.join(bigram)}, Frequency: {frequency}")

# Calculating the syllable frequencies

# Sorting the syllables by frequency in descending order
sorted_syllables = calculate_syllable_frequencies(syllable_list)

# Printing the top 20 syllable frequencies
print("\nTop 20 syllable frequencies:")
for syllable, frequency in sorted_syllables[:20]:
    print(f"Syllable: {''.join(syllable)}, Frequency: {frequency}")

# print("\nTop 20 syllable bigram frequencies:")
# for syllable, frequency in sorted_syllables[:20]:
#     print(f"Syllable: {''.join(syllable)}, Frequency: {frequency}")


bt [('आवेदन', ['आ', 'व्', 'ए', 'द्', 'अ', 'न्', 'अ']), ('करने', ['क्', 'अ', 'र्', 'अ', 'न्', 'ए']), ('की', ['क्', 'ई']), ('आखिरी', ['आ', 'ख्', 'इ', 'र्', 'ई']), ('तारीख', ['त्', 'आ', 'र्', 'ई', 'ख्', 'अ']), ('जनवरी', ['ज्', 'अ', 'न्', 'अ', 'व्', 'अ', 'र्', 'ई']), ('है।', ['ह्', 'ऐ'])]
uwl [['आ', 'व्', 'ए', 'द्', 'अ', 'न्', 'अ'], ['क्', 'अ', 'र्', 'अ', 'न्', 'ए'], ['क्', 'ई'], ['आ', 'ख्', 'इ', 'र्', 'ई'], ['त्', 'आ', 'र्', 'ई', 'ख्', 'अ'], ['ज्', 'अ', 'न्', 'अ', 'व्', 'अ', 'र्', 'ई'], ['ह्', 'ऐ']]
Top 20 characters:
Character: अ, Frequency: 8
Character: र्, Frequency: 4
Character: ई, Frequency: 4
Character: आ, Frequency: 3
Character: न्, Frequency: 3
Character: व्, Frequency: 2
Character: ए, Frequency: 2
Character: क्, Frequency: 2
Character: ख्, Frequency: 2
Character: द्, Frequency: 1
Character: इ, Frequency: 1
Character: त्, Frequency: 1
Character: ज्, Frequency: 1
Character: ह्, Frequency: 1
Character: ऐ, Frequency: 1

Top 20 bigram frequencies:
Bigram: अन्, Frequency: 3
Bigram: र्ई

In [ ]:

import sentencepiece as spm
from transformers import BertTokenizer
from transformers import AutoModel, AutoTokenizer
# Load the corpus from file

corpus_file = "hi_100.txt"

# Train Unigram model using SentencePiece
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix='unigram_model', model_type='unigram', vocab_size = 1000)
sp_unigram = spm.SentencePieceProcessor()
sp_unigram.load('unigram_model.model')
print("Unigram model trained successfully")

Unigram model trained successfully


In [57]:
import sentencepiece as spm

# Training our model using SentencePiece for vocab_size=1000
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix='bpe_model_1000', model_type='bpe', vocab_size=1000)

print("BPE model trained successfully")

BPE model trained successfully


In [ ]:

# Training our model using SentencePiece for vocab_size=2000
spm.SentencePieceTrainer.train(input=corpus_file, model_prefix='bpe_model_2000', model_type='bpe', vocab_size=2000)

print("BPE model trained successfully")

BPE model trained successfully


In [ ]:
from transformers import BertTokenizer, AutoTokenizer

tokenizer_mbert1 = BertTokenizer.from_pretrained('bert-base-multilingual-cased' , max_length=1000, truncation=True)


In [ ]:
tokenizer_mbert1 = BertTokenizer.from_pretrained('bert-base-multilingual-cased' , max_length=2000, truncation=True)


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")


In [66]:
def cbslist(unigram_tokens):
  # Remove leading '▁' from tokens and create unigrams
  unigram_op1 = [tokens.replace('▁', '') if tokens.startswith('▁') else tokens for tokens in unigram_tokens]
  unigram_op2 = [i for i in unigram_op1 if len(i) > 0 and (i[0] in List_matras or i[0] in List_swaras or i[0] in List_vyanjan)]
  broken_text = [(i,unicode_correction(i)) for i in unigram_op2]

  unicode_word_list = []

  # Printing the broken text along with the original words, syllables, and bigrams
  for word, unicode_word in broken_text:
      unicode_word_list.append(unicode_word)
      bigram_word = create_bigrams(unicode_word)
      syllable_list = form_syllable([unicode_word])  # Call form_syllable for the current word
      syllable_list_bigrams = create_bigrams(syllable_list)

      # print(f"Extracted Word: {word}:")
      # print(f"Unicode-corrected Characters: {unicode_word}")
      # print(f"Unicode Bigram: {bigram_word}")
      # print(f"Syllables: {syllable_list}")
      # print(f"Syllables Bigram: {syllable_list_bigrams}\n")

  print("uwl",unicode_word_list)
  # Calling the form_syllable function
  syllable_list = form_syllable(unicode_word_list)

  # Printing the syllables in a similar format as characters and bigrams
  syllable_bigram_list = [syllable_list[i]+syllable_list[i+1] for i in range(len(syllable_list)-1)]

  # Calculating the frequency of each character

  # Sorting the characters by frequency in descending order as we need the most freq char on top
  sorted_characters = calculate_character_frequencies(broken_text)
  # Calculating bigram frequencies
  bigram_freq = calculate_bigram_frequencies(broken_text)

  # Printing the top 20 unicode corrected characters and their frequencies
  print("Top 20 characters:")
  for char, frequency in sorted_characters[:20]:
      print(f"Character: {char}, Frequency: {frequency}")

  # Print the top 20 bigram and their frequencies
  print("\nTop 20 bigram frequencies:")
  for bigram, frequency in bigram_freq[:20]:
      print(f"Bigram: {''.join(bigram)}, Frequency: {frequency}")

  # Calculating the syllable frequencies

  # Sorting the syllables by frequency in descending order
  sorted_syllables = calculate_syllable_frequencies(syllable_list)

  # Printing the top 20 syllable frequencies
  print("\nTop 20 syllable frequencies:")
  for syllable, frequency in sorted_syllables[:20]:
      print(f"Syllable: {''.join(syllable)}, Frequency: {frequency}")


In [ ]:
# Assuming 'sp_unigram' is imported and defined properly
from transformers import BertTokenizer, AutoTokenizer
# Tokenize text
with open('sample.txt', "r", encoding="utf-8") as file:
    text = file.read()
unigram_tokens = sp_unigram.encode_as_pieces(text)

cbslist(unigram_tokens)

In [ ]:
from transformers import BertTokenizer

# Initialize the BPE tokenizer
tokenizer_bpe = BertTokenizer.from_pretrained("bert-base-uncased",max_length=1000)

# Tokenize text using BPE model
bpe_tokens = tokenizer_bpe.tokenize(text)

cbslist(bpe_tokens)

In [ ]:
from transformers import BertTokenizer
tokenizer_bpe_2000 = BertTokenizer.from_pretrained("bert-base-uncased",max_length=2000)
bpe_tokens = tokenizer_bpe_2000.tokenize(text)

cbslist(bpe_tokens)

In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-multilingual-cased",max_length=1000)

# Tokenize text
with open('sample.txt', "r", encoding="utf-8") as file:
    text = file.read()

# Tokenize the text
mbert_tokens = tokenizer_bert.tokenize(text)

cbslist(mbert_tokens)

In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer_bert_2000 = BertTokenizer.from_pretrained('bert-base-multilingual-cased',max_length=2000)

# Tokenize text
with open('sample.txt', "r", encoding="utf-8") as file:
    text = file.read()

# Tokenize the text
mbert_tokens = tokenizer_bert_2000.tokenize(text)

cbslist(mbert_tokens)

In [ ]:
from transformers import AutoTokenizer
tokenizer_indic = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

indicbert_tokens = tokenizer_indic.tokenize(tokenizer.decode(tokenizer.encode(text, max_length=1000, truncation=True)))

cbslist(indicbert_tokens)

In [ ]:
from transformers import AutoTokenizer
tokenizer_indic_2000 = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

indicbert_tokens = tokenizer_indic_2000.tokenize(tokenizer.decode(tokenizer.encode(text, max_length=2000, truncation=True)))

cbslist(indicbert_tokens)

In [ ]:
wst_tokens = text.split(" ")
cbslist(wst_tokens)

In [99]:
ans = [
    ['ढाबों में','रेट लिस्ट', 'जरूर'],
    ['ये गाना', 'पब और लाउंज के लिए', 'फिट बैठता है', 'इसलिए', 'मेकर्स ने', 'इसे पब में रिलीज करने का फैसला किया'],
    ['आप', 'यह पावरप्वॉइण्ट फाइल', 'कोकाकोला के चित्र', 'पर', 'क्लिक कर', 'डाउनलोड कर', 'सकते हैं'],
    ['मार्सिलिया (Marsilea) तथा सिरेटोप्टेरिस (Ceratopteris)', 'जैसे', 'टेरिडोफाइट्स का', 'उपयोग', 'सब्जी के रूप में', 'होता है'],
    ['बैठक के बाद', 'से', 'मीडिया में', 'कयास लगाए जाने लगे थे', 'कि', 'जम्मू-कश्मीर पर', 'केंद्र सरकार ने', 'कोई बड़ा फैसला', 'ले लिया है'],
    ['उन्होंने कहा', 'डॉ राजेन्द्र प्रसाद', 'बाबा साहेब भीमराव अंबेडकर', 'सरदार पटेल', 'मौलाना आजाद', 'सुचेता कृपलानी', 'और', 'अनेक अनगिनत महापुरुषों', 'ने', 'प्रत्यक्ष और अप्रत्यक्ष', 'योगदान', 'देकर', 'ये महान विरासत', 'हमें', 'सौंपी हैं'],
    ['अकसर', 'हम', 'ऐसे', 'सवालों से', 'जूझना', 'ज़रूरी', 'नहीं', 'समझते'],
    ['यह', 'उपयोगी था।', 'इस सुविधा', 'को', "'निष्क्रिय करने'", 'के बाद', 'चीजें', 'बहुत', 'स्पष्ट थीं', 'और', 'मैं', 'खाली पैकेजों', 'को', 'हटा सकता थ'],
    ['करण जौहर', 'ने', 'यह तस्वीर अपने', 'इंस्टाग्राम पर', 'शेयर', 'की है', 'जिसमें', 'आलिया भट्ट', 'उनके', 'बेटे', 'यश', 'को', 'राखी बांधती', 'नजर आ रही हैं'],
    ['जयपुर :', 'राजस्थान में', 'गहलोत मंत्रिमंडल', 'गठन में', 'लोकसभा चुनाव', 'की', 'छाया', 'रहेगी'],
    ['उन्होंने', 'मुख्य रूप से', 'प्रदेश की', 'सभी मार्गों को', 'गड्ढा मुक्त करने हेतु', 'आवश्यक मरम्मत कार्य', 'गुणवत्ता के साथ', 'शीघ्र पूर्ण करने', 'कहा'],
    ['थोक बाजार में', 'टमाटर', 'एक दिन में', '40 फीसदी तक', 'महंगा', 'हो रहा है'],
    ['उनके मुताबिक़', "''ऑक्सीजन की बात", 'किसी को', 'बताई नहीं गई थी', 'लेकिन', 'मेरी बच्ची के मुंह से', 'जब ख़ून निकलने लगा', 'तो', 'समझ में आया', 'कि', 'वो दम घुटने से मरी है'],
    ['नई दिल्ली:', 'टीम इंडिया के', 'उभरते बल्लेबाजी-ऑलराउंडर', 'हार्दिक पंड्या के', 'लिए दक्षिण अफ्रीकी दौरा', 'उनकी और करोड़ों भारतीय क्रिकेटप्रेमियों के लिए', 'निराशाजनक', 'साबित हुआ'],
    ['सवाल :', 'गोलकीपर सविता (पुनिया)', 'ने', 'टूर्नामेंट में', 'शानदार प्रदर्शन', 'किया'],
    ['इस संदर्भ में', 'कोई', 'पक्का साक्ष्य', 'नहीं', 'मिला है'],
    ['सूत्र बताते हैं', 'कि', 'अपनी बातचीत', 'को', 'अपलोड करने', 'का', 'कारण', 'यह', 'था', 'कि', 'इस तरह', 'कापड़ी', 'की', 'मक्खन-मालिश', 'हो जाएगी'],
    ['उन्होेंने बताया', 'कि', 'इन योजनाओं', 'के बन जाने से', 'जहां', '1250 हेक्टेयर', 'भूमि', 'सिंचित', 'होगी'],
    ['उसने तुरंत', 'बैंक में जाकर', 'पता किया', 'तो', 'उसके खाते से', 'किसी ने', 'एटीएम के माध्यम से', 'तीस हजार रुपये', 'निकाल लिए थे'],
    ['बाल्को', 'पॉट लाइन', 'प्रचालन प्रमुख', 'निकेत श्रीवास्तव', 'प्रभारी रूम-टू', 'पवन एम पाटिल', 'और', 'पॉट लाइन अनुरक्षण प्रमुख', 'दुर्गा प्रसाद पांडा', 'ने', 'बाल्कोकर्मियों', 'और', 'ठेका कामगारों', 'को', 'सम्मानित', 'किय'],
    ['क्या यह', 'प्रैक्टिकल है', 'या', 'सिर्फ', 'एक कहानी'],
    ['एनडीए में', 'बिहार के', 'मुख्यमंत्री', 'नीतीश कुमार', 'की', 'पार्टी', 'जनता दल यूनाइटेड', '(जेडीयू)', 'भी', 'शामिल', 'है'],
    ['नवविवाहित', 'प्रेमी जोड़ों को', 'सुरक्षित आश्रय स्थल', 'प्रदान करने के लिए', 'जेल परिसर में', 'सुरक्षित आश्रय', 'भवन की', 'व्यवस्था', 'की', 'गई है'],
    ['एएसआई जगदीश ने', 'बताया कि', 'युवक की जेब की तलाशी लेने पर', 'एक मोबाइल के अलावा', 'कुछ नहीं मिला है'],
    ['गुरुग्राम में', 'साइकिल रैली में', 'हिस्सा लेते', 'अशोक तंवर']
]
pre = ["ढाबों में रेट लिस्ट जरूर", "ये गाना पब और लाउंज के लिए फिट बैठता है इसलिए मेकर्स ने इसे पब में रिलीज करने का फैसला किया", "आप यह पावरप्वॉइण्ट फाइल कोकाकोला के चित्र पर क्लिक कर डाउनलोड कर सकते हैं", "मार्सिलिया (Marsilea) तथा सिरेटोप्टेरिस (Ceratopteris) जैसे टेरिडोफाइट्स का उपयोग सब्जी के रूप में होता है", "बैठक के बाद से मीडिया में कयास लगाए जाने लगे थे कि जम्मू-कश्मीर पर केंद्र सरकार ने कोई बड़ा फैसला ले लिया है", "उन्होंने कहा, डॉ राजेन्द्र प्रसाद, बाबा साहेब भीमराव अंबेडकर, सरदार पटेल, मौलाना आजाद, सुचेता कृपलानी और अनेक अनगिनत महापुरुषों ने प्रत्यक्ष और अप्रत्यक्ष योगदान देकर ये महान विरासत हमें सौंपी हैं", "अकसर हम ऐसे सवालों से जूझना ज़रूरी नहीं समझते", "यह उपयोगी था। इस सुविधा को 'निष्क्रिय करने' के बाद चीजें बहुत स्पष्ट थीं और मैं खाली पैकेजों को हटा सकता थ", "करण जौहर ने यह तस्वीर अपने इंस्टाग्राम पर शेयर की है, जिसमें आलिया भट्ट उनके बेटे यश को राखी बांधती नजर आ रही हैं", "जयपुर : राजस्थान में गहलोत मंत्रिमंडल गठन में लोकसभा चुनाव की छाया रहेगी", "उन्होंने मुख्य रूप से प्रदेश की सभी मार्गों को गड्ढा मुक्त करने हेतु आवश्यक मरम्मत कार्य गुणवत्ता के साथ शीघ्र पूर्ण करने कहा", "थोक बाजार में टमाटर एक दिन में 40 फीसदी तक महंगा हो रहा है", "उनके मुताबिक़, ''ऑक्सीजन की बात किसी को बताई नहीं गई थी, लेकिन मेरी बच्ची के मुंह से जब ख़ून निकलने लगा तो समझ में आया कि वो दम घुटने से मरी है", 'नई दिल्ली: टीम इंडिया के उभरते बल्लेबाजी-ऑलराउंडर हार्दिक पंड्या के लिए दक्षिण अफ्रीकी दौरा उनकी और करोड़ों भारतीय क्रिकेटप्रेमियों के लिए निराशाजनक साबित हुआ.', 'सवाल : गोलकीपर सविता (पुनिया) ने टूर्नामेंट में शानदार प्रदर्शन किया', 'इस संदर्भ में कोई पक्का साक्ष्य नहीं मिला है', 'सूत्र बताते हैं कि अपनी बातचीत को अपलोड करने का कारण यह था कि इस तरह कापड़ी की मक्खन-मालिश हो जाएगी।', 'उन्होेंने बताया कि इन योजनाओं के बन जाने से जहां 1250 हेक्टेयर भूमि सिंचित होगी', "उसने तुरंत बैंक में जाकर पता किया तो उसके खाते से किसी ने एटीएम के माध्यम से तीस हजार रुपये निकाल लिए थे", "बाल्को के पॉट लाइन प्रचालन प्रमुख निकेत श्रीवास्तव, प्रभारी रूम-टू पवन एम पाटिल और पॉट लाइन अनुरक्षण प्रमुख दुर्गा प्रसाद पांडा ने बाल्कोकर्मियों और ठेका कामगारों को सम्मानित किया", "क्या यह प्रैक्टिकल है या सिर्फ एक कहानी", "एनडीए में बिहार के मुख्यमंत्री नीतीश कुमार की पार्टी जनता दल यूनाइटेड (जेडीयू) भी शामिल है", "नवविवाहित प्रेमी जोड़ों को सुरक्षित आश्रय स्थल प्रदान करने के लिए जेल परिसर में सुरक्षित आश्रय भवन की व्यवस्था की गई है", "एएसआई जगदीश ने बताया कि युवक की जेब की तलाशी लेने पर एक मोबाइल के अलावा कुछ नहीं मिला है", "गुरुग्राम में साइकिल रैली में हिस्सा लेते अशोक तंवर"]

In [117]:
def calc(ans, unigram_t):
    tp = fp = fn = 0

    for true_tokens, pred_tokens in zip(ans, unigram_t):
        true_set = set(true_tokens)
        pred_set = set(clean(pred_tokens))

        tp += len(true_set.intersection(pred_set))
        fp += len(pred_set - true_set)
        fn += len(true_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f_score

unigram_t=[]
for line in pre:
  unigram_t.append(sp_unigram.encode_as_pieces(line))
p, r, f = calc(ans, unigram_t)
print("Precision:", p)
print("Recall:", r)
print("F-score:", f)


Precision: 0.0023752969121140144
Recall: 0.009259259259259259
F-score: 0.003780718336483932


In [118]:
bpe_t_1000=[]
for line in pre:
  bpe_t_1000.append(tokenizer_bpe.tokenize(line))
p, r, f = calc(ans, bpe_t_1000)
print("Precision:", p)
print("Recall:", r)
print("F-score:", f)


Precision: 0.0
Recall: 0.0
F-score: 0


In [122]:
bpe_t_2000=[]
for line in pre:
  bpe_t_2000.append(tokenizer_bpe_2000.tokenize(line))

p, r, f = calc(ans, bpe_t_2000)

print("Precision:", p)
print("Recall:", r)
print("F-score:", f)

Precision: 0.0
Recall: 0.0
F-score: 0


In [104]:
bert_t_1000=[]
for line in pre:
  bert_t_1000.append(tokenizer_bert.tokenize(line))
p, r, f = calc(ans, bert_t_1000)
print("Precision:", p)
print("Recall:", r)
print("F-score:", f)

Precision: 0.059233449477351915
Recall: 0.2361111111111111
F-score: 0.09470752089136489


In [103]:
bert_t_2000=[]
for line in pre:
  bert_t_2000.append(tokenizer_bert_2000.tokenize(line))
p, r, f = calc(ans, bert_t_2000)
print("Precision:", p)
print("Recall:", r)
print("F-score:", f)

Precision: 0.059233449477351915
Recall: 0.2361111111111111
F-score: 0.09470752089136489


In [108]:
ibert_t_1000=[]
for line in pre:
  ibert_t_1000.append(tokenizer_bert.tokenize(line))
p, r, f = calc(ans, ibert_t_1000)
print("Precision:", p)
print("Recall:", r)
print("F-score:", f)

Precision: 0.059233449477351915
Recall: 0.2361111111111111
F-score: 0.09470752089136489


In [109]:
ibert_t_2000=[]
for line in pre:
  ibert_t_2000.append(tokenizer_bert_2000.tokenize(line))
p, r, f = calc(ans, ibert_t_2000)
print("Precision:", p)
print("Recall:", r)
print("F-score:", f)

Precision: 0.059233449477351915
Recall: 0.2361111111111111
F-score: 0.09470752089136489


In [110]:
wst=[]
for line in pre:
  wst.append(line.split(" "))
p, r, f = calc(ans, wst)
print("Precision:", p)
print("Recall:", r)
print("F-score:", f)


Precision: 0.18793503480278423
Recall: 0.375
F-score: 0.25038639876352403
